# Counterfactual Group Identification

## Python Setup

In [1]:
import requests
import time
import pandas as pd
from pandas import Series
pd.options.display.max_rows = 100
import math
import numpy as np
from json import JSONDecodeError
from datetime import datetime
import gender_guesser.detector as gender

## Importing ID Replacer

In [2]:
id_replacer = pd.read_csv('../data/id_replacer.csv')

In [3]:
def id_replace(df):
    df = pd.merge(df, id_replacer, how='left', left_on='rsr_id', right_on='old_id')
    df['rsr_id'] = np.where(df['new_id'].notnull(), df['new_id'], df['rsr_id'])
    del df['old_id'], df['new_id']
    return df

## Connecting to API

For full documentation: https://docs.dimensions.ai/dsl/1.8.0/

The API Query functions are defined in the following code:

In [4]:
from api_query import execute_query
from api_query import pull_data

### API Parameters

In [5]:
# Filter operator 'in' requires 0 < items < 512
max_in_items = 100

# Limit exceeds maximum allowed limit 1000
max_return = 1000

# Offset cannot exceed 50000
max_overall_returns = 50000

## Grant and Publication Parsing Function

In [6]:
def parse_grant(full_resp):
    
    rsr_id = []
    grant_id = []
    funder_name = []
    funder_id = []
    rcdc_codes = []
    rcdc_names = []
    start_dates = []
    end_dates = []
    funding_amount = []
    for grant in full_resp:
        if 'RCDC' not in grant:
            grant['RCDC'] = [{'id':np.nan, 'name':np.nan}]
        if 'researchers' not in grant:
            grant['researchers'] = [{'id':np.nan}]
        for researcher in grant['researchers']: 
            if 'id' not in researcher:
                tmp = researcher
                grant['researchers'].append({'id':tmp})
        if 'start_date' not in grant:
            grant['start_date'] = np.nan
        if 'end_date' not in grant:
            grant['end_date'] = np.nan
        if 'funding_usd' not in grant:
            grant['funding_usd'] = np.nan
        for i in range(len(grant['researchers'])-1, -1, -1):
            if 'id' not in grant['researchers'][i]:
                del grant['researchers'][i]
        if 'funders' not in grant:
            grant['funders'] = [{'id':np.nan,'name':np.nan}]
        for researcher in grant['researchers']:
            for funder in grant['funders']:
                rsr_id.append(researcher['id'])
                grant_id.append(grant['id'])
                funder_id.append(funder['id'])
                funder_name.append(funder['name'])
                start_dates.append(grant['start_date'])
                end_dates.append(grant['end_date'])
                funding_amount.append(float(grant['funding_usd']))
                rcdc_codes_t = []
                rcdc_names_t = []
                for rcdc in grant['RCDC']:
                    rcdc_codes_t.append(str(rcdc['id']))
                    rcdc_names_t.append(str(rcdc['name']))
                rcdc_codes.append(";".join(rcdc_codes_t))
                rcdc_names.append(";".join(rcdc_names_t))
                
    grnts = pd.DataFrame({'rsr_id':rsr_id
                          , 'grant_id':grant_id
                          , 'funder_name':funder_name
                          , 'funder_id':funder_id
                          , 'rcdc_codes':rcdc_codes
                          , 'rcdc_names':rcdc_names
                          , 'start_date':start_dates
                          , 'end_date':end_dates
                          , 'funding_amount':funding_amount
                         })

    return grnts

In [7]:
def parse_publication(full_resp):

    pub_id = []
    pmid = []
    dates = []
    citations = []
    dois = []
    supporting_grants = []
    nb_authors = []
    author_id = []
    author_country = []
    author_affiliation = []
    author_affiliation_id = []

    for pub in full_resp:
        if 'id' not in pub:
            pub['id'] = np.nan
        if 'pmid' not in pub:
            pub['pmid'] = np.nan
        if 'author_affiliations' not in pub:
            pub['author_affiliations'] = [[]]
        for author in pub['author_affiliations'][0]: 
            if 'researcher_id' not in author:
                author['researcher_id'] = np.nan
            if ('affiliations' not in author)  | (len(author['affiliations'])==0):
                author['affiliations'] = [{'country_code':np.nan, 'name': np.nan, 'id':np.nan}]
            for affiliation in author['affiliations']:
                if 'country_code' not in affiliation:
                    affiliation['country_code']=np.nan
                if 'name' not in affiliation:
                    affiliation['name']=np.nan
                if 'id' not in affiliation:
                    affiliation['id']=np.nan
        if 'date' not in pub:
            pub['date'] = np.nan
        if 'times_cited' not in pub:
            pub['times_cited'] = np.nan
        if 'doi' not in pub:
            pub['doi'] = np.nan
        if 'supporting_grant_ids' not in pub:
            pub['supporting_grant_ids'] = [np.nan]
        nb = len(pub['author_affiliations'][0])
        for author in pub['author_affiliations'][0]:
            nb_authors.append(float(nb))
            author_id.append(author['researcher_id'])
            author_country.append(author['affiliations'][0]['country_code'])
            author_affiliation.append(author['affiliations'][0]['name'])
            author_affiliation_id.append(author['affiliations'][0]['id'])
            pub_id.append(pub['id'])
            dates.append(pub['date'])
            citations.append(float(pub['times_cited']))
            dois.append(pub['doi'])
            pmid.append(pub['pmid'])
            supporting_grants.append(pub['supporting_grant_ids'])

    pubs = pd.DataFrame({'pub_id':pub_id
                         , 'pmid': pmid
                         , 'date':dates
                         , 'doi':dois
                         , 'citations':citations
                         , 'nb_authors':nb_authors
                         , 'rsr_id':author_id
                         , 'rsr_country':author_country
                         , 'rsr_affiliation':author_affiliation
                         , 'rsr_affiliation_id':author_affiliation_id
                         , 'supporting_grants':supporting_grants
                        })
    pubs['supporting_grants'] = pubs['supporting_grants'].apply(lambda x: np.nan if x == [np.nan] else  ';'.join(x))
    
    return pubs

In [8]:
def parse_rcdc(full_resp, data_type):
    
    data_id = []
    rcdc_codes = []
    rcdc_names = []
    for elem in full_resp:
        if 'id' not in elem:
            elem['id'] = np.nan
        if 'RCDC' not in elem:
            elem['RCDC'] = [{'id':np.nan, 'name':np.nan}]
        for rcdc in elem['RCDC']:
            if rcdc['name'] != np.nan:
                data_id.append(elem['id'])
                rcdc_names.append(str(rcdc['name']))

    rcdc = pd.DataFrame({data_type+'_id':data_id
                         , 'rcdc_name':rcdc_names})
    rcdc['rcdc_name'] = rcdc['rcdc_name'].str.strip().str.upper()

    return rcdc

## Counterfactual based on Funding Agencies

### Define Comparison Agencies:

Funding Agencies considered are NIH Wellcome Trust, Australian  NRMC, MRC in UK

In [9]:
funders = ['Wellcome Trust', 'National Health and Medical Research Council', 'Cancer Research UK', 
           'National Cancer Institute']

### Pull all Researchers Funded by these Agencies

In [ ]:
string = "search grants where funders.name in [{}] and start_year>=2007 and start_year<=2012"
string += " return grants[id+researchers+RCDC+funders+start_date+end_date+funding_usd]"
full_resp = pull_data(string=string, in_list=funders, in_type='funding agencies', return_type='grants',
                      max_in_items=max_in_items, max_return=max_return, max_overall_returns=max_overall_returns)

In [ ]:
funded_grnts = parse_grant(full_resp).drop_duplicates().reset_index(drop=True)

funded_grnts_rcdc = parse_rcdc(full_resp, data_type='grant')

In [ ]:
# Remove all grants with no researcher info
funded_grnts = funded_grnts[funded_grnts['rsr_id'].notnull()]

# Remove all other funders that might have appeared
funded_grnts = funded_grnts[(funded_grnts['funder_name'].isin(funders))]

# Restrict to grants funded between 2007 and 2012:
funded_grnts = funded_grnts[(pd.DatetimeIndex(funded_grnts['start_date']).year>=2007)
                          & (pd.DatetimeIndex(funded_grnts['start_date']).year<=2012)].reset_index(drop=True)

Some of the counterfactual agencies (NCI, Cancer Research UK) are cancer-specific funders. Others however, such as the Wellcome Trust and the NHMRC, fund different types of medical research. For this second category of non-cancer-specific funders, we must restrict to grants on the topic of Cancer. I use the RCDC codes to restrict to Cancer-related grants.

In [ ]:
# For non-cancer-specific funders, restrict to Cancer Grants:
general_funders = ['Wellcome Trust', 'National Health and Medical Research Council']
funded_grnts = funded_grnts[(funded_grnts['funder_name'].isin(general_funders)==False)
                            |(funded_grnts['rcdc_names'].str.upper().str.contains("CANCER"))].reset_index(drop=True)

In [ ]:
funded_grnts['funder_name'].value_counts()

In [ ]:
funded_grnts.head()

In [ ]:
funded_grnts = id_replace(funded_grnts)

In [ ]:
funded_grnts.to_csv('../data/counterfactual_funded_grants.csv', index=False)
funded_grnts_rcdc.to_csv('../data/topic_lookups/counterfactual_funded_grants_rcdc.csv', index=False)

### Pull all Grants, Publications, and Personal Information from these Researchers

In [10]:
funded_grnts = pd.read_csv('../data/counterfactual_funded_grants.csv')
rsrs = list(funded_grnts['rsr_id'].drop_duplicates())
print(len(rsrs))

13026


#### Grants

In [ ]:
string = "search grants where researchers.id in [{}]"
string += " return grants[id+researchers+RCDC+funders+start_date+end_date+funding_usd]"
full_resp = pull_data(string=string, in_list=rsrs, in_type='researchers', return_type='grants'
                      , max_in_items=max_in_items, max_return=max_return, max_overall_returns=max_overall_returns)

In [ ]:
grnts = parse_grant(full_resp).drop_duplicates().reset_index(drop=True)

grnts_rcdc = parse_rcdc(full_resp, data_type='grant')

In [ ]:
grnts.describe(include='all')

In [ ]:
grnts = id_replace(grnts)

In [ ]:
grnts.to_csv('../data/counterfactual_researcher_grants.csv', index=False)
grnts_rcdc.to_csv('../data/topic_lookups/counterfactual_researcher_grants_rcdc.csv', index=False)

#### Publications

In [ ]:
string = "search publications where researchers.id in [{}]"
string += " return publications[id+doi+pmid+author_affiliations+date+supporting_grant_ids+times_cited+RCDC]"
full_resp = pull_data(string=string, in_list=rsrs, in_type='researchers', return_type='publications', 
                      max_in_items=max_in_items, max_return=max_return, max_overall_returns=max_overall_returns)

In [ ]:
pubs = parse_publication(full_resp).drop_duplicates().reset_index(drop=True)

pubs_rcdc = parse_rcdc(full_resp, data_type='pub')

In [ ]:
pubs.describe(include='all')

In [ ]:
pubs = id_replace(pubs)

In [ ]:
# Publication IDs:
pub_ids = pubs[pubs['pmid'].notnull()][['rsr_id', 'pmid']].drop_duplicates()

In [ ]:
pubs.to_csv('../data/counterfactual_researcher_publications.csv', index=False)
pubs_rcdc.to_csv('../data/topic_lookups/counterfactual_researcher_publications_rcdc.csv', index=False)
pub_ids.to_csv('../data/counterfactual_pub_ids.csv', index=False)

#### Personal Information

In [45]:
first_name = []
last_name = []
first_affiliation = []
first_country = []
first_pub_year = []

for rsr in rsrs:
    
    print("Querying {} out of {} reseachers...".format(rsrs.index(rsr)+1, len(rsrs)), end = '\r')
        
    # Pull Name:
    time.sleep(2)
    stop = False
    string = "search grants where researchers.id in [\"{}\"]".format(rsr)
    string += " return researchers"
    resp = execute_query(string)
    for elem in resp['researchers']:
        if elem['id'] == rsr:
            stop = True
            if 'first_name' in elem:
                first_name.append(elem['first_name'])
            else:
                first_name.append(np.nan)
            if 'last_name' in elem:
                last_name.append(elem['last_name'])
            else:
                last_name.append(np.nan)
            break
    if stop==False:
        first_name.append(np.nan)
        last_name.append(np.nan)
    
    # Number of publications:
    string = "search publications where researchers.id in [\"{}\"] and date is not empty".format(rsr)
    string += " return publications sort by date limit 1"
    resp = execute_query(string)
    if resp == "RESPONSE ERROR":
        first_pub_year.append(np.nan)
        first_country.append(np.nan)
        first_affiliation.append(np.nan)
        continue
    nb_pubs = resp['_stats']['total_count']

    # Pull First Country, Affiliation, and Pub Date:    
    time.sleep(2)
    stop = False
    if nb_pubs == 0:
        first_pub_year.append(np.nan)
        first_country.append(np.nan)
        first_affiliation.append(np.nan)
        continue
    string = "search publications where researchers.id in [\"{}\"] and date is not empty".format(rsr)
    string += " return publications[author_affiliations+date] sort by date"
    string += " limit 500 skip {}".format(max(nb_pubs-500, 0))
    resp = execute_query(string)
    if resp == "RESPONSE ERROR":
        first_pub_year.append(np.nan)
        first_country.append(np.nan)
        first_affiliation.append(np.nan)
        continue
    first_pub_year.append(datetime.strptime(resp['publications'][len(resp['publications'])-1]['date'], 
                                            '%Y-%m-%d').year)
    for i in range(len(resp['publications']), 0, -1):
        for elem in resp['publications'][i-1]['author_affiliations'][0]:
            if 'researcher_id' in elem:
                if elem['researcher_id'] == rsr:
                    if len(elem['affiliations'])>0:
                        stop = True
                        if 'country_code' in elem['affiliations'][0]:
                            first_country.append(elem['affiliations'][0]['country_code'])
                        else:
                            first_country.append(np.nan)
                        if 'name' in elem['affiliations'][0]:
                            first_affiliation.append(elem['affiliations'][0]['name'])
                        else:
                            first_affiliation.append(np.nan)
                    break
        if stop==True:
            break
    if stop==False:
        first_country.append(np.nan)
        first_affiliation.append(np.nan)
            
print("\nDone !")

rsr_info = pd.DataFrame({'rsr_id':rsrs
                         , 'first_name':first_name
                         , 'last_name': last_name
                         , 'affiliation': first_affiliation
                         , 'country': first_country
                         , 'career_start': first_pub_year
                        })

Querying 479 out of 13026 reseachers...
continue



KeyboardInterrupt: 

In [ ]:
rsr_info = pd.DataFrame({'rsr_id':rsrs
                         , 'first_name':first_name
                         , 'last_name': last_name
                         , 'affiliation': first_affiliation
                         , 'country': first_country
                         , 'career_start': first_pub_year
                        })

In [ ]:
rsr_info.head()

In [ ]:
# Gender Imputer
d = gender.Detector()

# Gender Abbreviation Table
gender_abbr = pd.DataFrame({'gender': ['male', 'mostly_male', 'unknown', 'andy', 'mostly_female', 'female']
                            , 'gender_abbr': ['M', 'M', 'UNKNOWN', 'UNKNOWN', 'F', 'F']})

In [ ]:
rsr_info['gender'] = rsr_info['first_name'].apply(lambda x: d.get_gender(x))
rsr_info = pd.merge(rsr_info, gender_abbr, how='left', on='gender')
del rsr_info['gender']
rsr_info.rename(columns={'gender_abbr':'gender'}, inplace=True)

In [ ]:
rsr_info['gender'].value_counts(normalize=True)

In [ ]:
rsr_info.to_csv('../data/counterfactual_researcher_info.csv', index=False)

## Sandbox